In [ ]:
#import torch
from pytorch3d.io import load_obj
from vtk import vtkPolyDataNormals,vtkPolyDataReader, vtkPolyData
import os
from torch import int64, float32, clamp, tensor
from vtk.util.numpy_support import vtk_to_numpy
from monai.transforms import (
    ToTensor
)
from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene
from pytorch3d.structures import Meshes
import matplotlib.pyplot as plt
import numpy as np

: 

In [ ]:
def ComputeNormals(surf):
    normals = vtkPolyDataNormals()
    normals.SetInputData(surf);
    normals.ComputeCellNormalsOff();
    normals.ComputePointNormalsOn();
    normals.SplittingOff();
    normals.Update()
    
    return normals.GetOutput()


def ReadSurf(fileName):
    fname, extension = os.path.splitext(fileName)
    reader = vtkPolyDataReader()
    reader.SetFileName(fileName)
    reader.Update()
    surf = reader.GetOutput()
    return surf

def ScaleSurf(surf, mean_arr = None, scale_factor = None):
    surf_copy = vtkPolyData()
    surf_copy.DeepCopy(surf)
    surf = surf_copy

    shapedatapoints = surf.GetPoints()

    #calculate bounding box
    mean_v = [0.0] * 3
    bounds_max_v = [0.0] * 3

    bounds = shapedatapoints.GetBounds()

    mean_v[0] = (bounds[0] + bounds[1])/2.0
    mean_v[1] = (bounds[2] + bounds[3])/2.0
    mean_v[2] = (bounds[4] + bounds[5])/2.0
    bounds_max_v[0] = max(bounds[0], bounds[1])
    bounds_max_v[1] = max(bounds[2], bounds[3])
    bounds_max_v[2] = max(bounds[4], bounds[5])

    shape_points = []
    for i in range(shapedatapoints.GetNumberOfPoints()):
        p = shapedatapoints.GetPoint(i)
        shape_points.append(p)
    shape_points = np.array(shape_points)
    
    #centering points of the shape
    if mean_arr is None:
        mean_arr = np.array(mean_v)
    # print("Mean:", mean_arr)
    shape_points = shape_points - mean_arr

    #Computing scale factor if it is not provided
    if(scale_factor is None):
        bounds_max_arr = np.array(bounds_max_v)
        scale_factor = 1/np.linalg.norm(bounds_max_arr - mean_arr)

    #scale points of the shape by scale factor
    # print("Scale:", scale_factor)
    shape_points_scaled = np.multiply(shape_points, scale_factor)

    #assigning scaled points back to shape
    for i in range(shapedatapoints.GetNumberOfPoints()):
       shapedatapoints.SetPoint(i, shape_points_scaled[i])    

    surf.SetPoints(shapedatapoints)

    return surf, mean_arr, scale_factor

def ComputeNormals(surf):
    normals = vtkPolyDataNormals()
    normals.SetInputData(surf);
    normals.ComputeCellNormalsOff();
    normals.ComputePointNormalsOn();
    normals.SplittingOff();
    normals.Update()
    
    return normals.GetOutput()

: 

In [ ]:
path = '/Users/luciacev-admin/Desktop/ALIDDM/ALIDDM/random_crowm/T1_01_L_segmented.vtk'

surf = ReadSurf(path)
surf, mean_arr, scale_factor= ScaleSurf(surf) 
surf = ComputeNormals(surf) 
verts = ToTensor(dtype=float32)(vtk_to_numpy(surf.GetPoints().GetData()))
faces = tensor(vtk_to_numpy(surf.GetPolys().GetData()).reshape(-1, 4)[:,1:])
region_id = ToTensor(dtype=int64 )(vtk_to_numpy(surf.GetPointData().GetScalars("PredictedID")))
region_id = clamp(region_id, min=0)
print(region_id,type(region_id))
print(verts, type(verts),verts.size())




mesh = Meshes(verts = [verts], faces = [faces]) #, textures = region_id)



fig = plot_scene({
    "subplot1": {
        "cow_mesh": mesh
    }
})
fig.show()

: 